In [63]:
import pandas as pd

In [64]:
df = pd.read_csv('clinical.csv')

In [65]:
df

,ID,SEX,MS_TYPE,AGE,EDSS,DATASET,DIAG_YEARS,BMI,THERAPY,outliers_V1,outliers_V2
0,p001,M,primary_progressive,40,7,2,11,19.918,Functional_electric_stimulation,0,14
1,p002,M,secondary_progressive,44,7,2,17,20.529,Motor_Program_Activating_Therapy,4,0
2,p003,M,primary_progressive,51,6,2,21,25.249,Functional_electric_stimulation,4,0
3,p004,M,relapsing_remitting,29,3,2,9,21.500,Functional_electric_stimulation,4,2
4,p005,F,secondary_progressive,60,6,2,21,21.411,Functional_electric_stimulation,0,4
5,p006,M,secondary_progressive,68,4,2,7,21.605,Motor_Program_Activating_Therapy,2,3
6,p007,F,relapsing_remitting,36,4,2,9,19.370,Functional_electric_stimulation,6,7
7,p008,F,relapsing_remitting,32,4,2,4,19.141,Functional_electric_stimulation,5,7
8,p009,F,relapsing_remitting,35,4,1,20,16.436,Motor_Program_Activating_Therapy,3,0
9,p010,M,secondary_progressive,54,6,2,12,23.148,Motor_Program_Activating_Therapy,20,6


In [66]:
col_names = lambda number_of_cols: [str(i) for i in range(number_of_cols)]

In [88]:
fc = pd.read_csv('FC\p001_1.csv', names = col_names(116))

In [89]:
fc

,0,1,2,3,4,5,6,7,8,9,...,106,107,108,109,110,111,112,113,114,115
0,1.000000,0.817564,0.204598,-0.078401,0.289494,0.090226,0.497704,0.089581,0.455409,0.053830,...,-0.072903,0.085836,-0.106347,-0.119842,-0.090812,0.042023,0.125418,-0.086712,-0.188296,-0.141228
1,0.817564,1.000000,0.254734,-0.085640,0.206634,0.032366,0.317668,0.094261,0.288953,0.005228,...,-0.157830,0.037733,-0.156203,-0.003416,-0.070900,-0.019105,0.062476,-0.051509,-0.160459,-0.176699
2,0.204598,0.254734,1.000000,0.418168,0.777710,0.337238,0.304956,0.337601,0.509011,0.358025,...,-0.035532,0.075027,-0.274796,-0.111992,-0.074269,-0.035510,0.064830,0.088095,0.248809,-0.259217
3,-0.078401,-0.085640,0.418168,1.000000,0.350748,0.794528,0.133066,0.378738,0.095346,0.442440,...,-0.195316,-0.066236,-0.181326,-0.076110,-0.161506,-0.148111,-0.069177,-0.020364,0.212961,-0.239621
4,0.289494,0.206634,0.777710,0.350748,1.000000,0.507826,0.553142,0.469009,0.713740,0.574525,...,0.022237,0.192382,-0.249915,-0.193518,-0.165759,0.023005,0.140473,0.108519,0.172550,-0.317077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,0.042023,-0.019105,-0.035510,-0.148111,0.023005,-0.172136,0.016610,-0.291438,-0.037145,-0.178627,...,-0.017592,0.304012,0.171264,0.045451,0.489263,1.000000,0.691592,0.286902,0.245557,-0.164185
112,0.125418,0.062476,0.064830,-0.069177,0.140473,-0.066660,0.139559,-0.105685,0.101847,-0.035088,...,0.122634,0.442138,0.079924,-0.119296,0.237733,0.691592,1.000000,0.481911,0.221168,-0.145863
113,-0.086712,-0.051509,0.088095,-0.020364,0.108519,-0.054478,0.078251,0.084112,0.101845,0.078863,...,0.198316,0.199112,0.129565,0.066621,0.114709,0.286902,0.481911,1.000000,0.438909,-0.277216
114,-0.188296,-0.160459,0.248809,0.212961,0.172550,0.096771,0.003083,0.178732,0.016812,0.156808,...,-0.058638,0.086031,-0.011735,0.100650,0.177972,0.245557,0.221168,0.438909,1.000000,-0.202847


In [92]:
def AdjacencyMatrixBuilder(dataframe):
    dataframe = dataframe.abs() 
    dataframe[dataframe == 1.] = 0 # maybe there is a better solution to set the diagonal vals = 0
    dataframe[dataframe >= 0.3] = 1
    dataframe[dataframe < 0.3] = 0
    return dataframe.astype(int)

In [93]:
am = AdjacencyMatrixBuilder(fc)
am

,0,1,2,3,4,5,6,7,8,9,...,106,107,108,109,110,111,112,113,114,115
0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
112,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
113,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [75]:
import json
from pandas.io.json import json_normalize

with open('DATS.json', encoding="utf8") as json_data:
    data = json.load(json_data)

<function dict.keys>

In [79]:
data.keys()

dict_keys(['title', 'description', 'identifier', 'dates', 'creators', 'types', 'version', 'privacy', 'licenses', 'distributions', 'primaryPublications', 'dimensions', 'acknowledges', 'keywords', 'extraProperties'])

In [86]:
data['dimensions']

[{'name': {'value': 'FC'},
  'description': 'Contains functional connectivity matrices as *.csv for each subject and visit. Also contains file AAL_labels.csv with labels of AAL atlas regions.'},
 {'name': {'value': 'nifti'},
  'description': 'The directory contains CONN preprocessed fMRI time-series. Each subject was recorded twice - before and after neuroproprioceptive "facilitation and inhibition" treatment.'},
 {'name': {'value': 'quality measures'},
  'description': 'Contains files with the appropriate displacement measures for each subject and visit.'}]